In [2]:
import json
import numpy as np

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
with open("../drawing/drawing_config.json", "r") as f:
    config = json.load(f)

T, u, v, normal, origin = np.array(config["T"]), np.array(config["u"]), np.array(config["v"]), np.array(config["normal"]), np.array(config["origin"])
poses = config["poses"]

def arrow(start, vec, color, name=""):
    return go.Scatter3d(
        x=[start[0], start[0] + vec[0]],
        y=[start[1], start[1] + vec[1]],
        z=[start[2], start[2] + vec[2]],
        mode="lines+markers",
        line=dict(color=color, width=4),
        marker=dict(size=3, color=color),
        name=name
    )

def plot_poses(poses, fig):

    # pose positions
    for i, s in enumerate(poses):
        pos = np.array([s["ee.x"], s["ee.y"], s["ee.z"]])
        rotvec = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])

        # --- Convert to rotation matrix ---
        rot = R.from_rotvec(rotvec)
        R_mat = rot.as_matrix()

        # --- Original coordinate axes ---
        x_axis = np.array([1, 0, 0])
        y_axis = np.array([0, 1, 0])
        z_axis = np.array([0, 0, 1])

        # --- Rotated axes ---
        x_rot = R_mat @ x_axis
        y_rot = R_mat @ y_axis
        z_rot = R_mat @ z_axis
        fig.add_trace(arrow(pos, x_rot * 0.01, 'red'))
        fig.add_trace(arrow(pos, y_rot * 0.01, 'blue'))
        fig.add_trace(arrow(pos, z_rot * 0.01, 'green'))

fig = go.Figure()

# Axes
fig.add_trace(arrow(origin, u * 0.01, "red", "u"))
fig.add_trace(arrow(origin, v * 0.01, "green", "v"))
fig.add_trace(arrow(origin, normal * 0.01, "black", "normal"))

plot_poses(poses, fig)
    # Layout
r = 0.1
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
        yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
        zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="Interactive TCP Visualization",
)
fig.show()